<a href="https://colab.research.google.com/github/pulseq/MR-Physics-with-Pulseq/blob/main/tutorials/03_k_space_sampling/notebooks/03_radial_center_out_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 3: Radial sequences, center-out (solutions)

---
Solutions
---
---

- **In the sequence code below, the current implementation is the solution after completing exercise 1. Changes are marked as comments starting with "Ex 1.".**
- **The solution for exercise 2 (ramp-sampled center-out radial) is in a separate notebook.**

---
&nbsp;
---

In this exercise we will modify the standard gradient echo radial sequence that is provided by (py)Pulseq. You can find the exercises at the bottom of this notebook!

# Initialization

In [ ]:
import importlib
if importlib.util.find_spec('pypulseq') is None:
    !pip install -q mrzerocore git+https://github.com/imr-framework/pypulseq.git

import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

import pypulseq as pp

!wget -nc https://raw.githubusercontent.com/pulseq/MR-Physics-with-Pulseq/main/utils/utils.py
from utils import animate, simulate_2d, recon_nufft_2d, plot_nd

# **Create, simulate, and reconstruct sequence**

**Radial GRE Sequence definition**

In [ ]:
plot = False
write_seq = False
animate_sequence = True

seq_filename = "gre_radial.seq"

# ======
# SETUP
# ======
fov = 256e-3
Nx = 64  # Define FOV and resolution
alpha = 30  # Flip angle
slice_thickness = 5e-3  # Slice thickness
# Ex 1. Removed delay_TE block (so we no longer need an echo time definition)
# TE = 3.5e-3  # Echo time
TR = 7.5e-3  # Repetition time

# Ex 1. Doubled number of spokes (removed division by 2), and updated to match solution of radial exercise
Nr = math.ceil(Nx * np.pi) - 1  # Number of radial spokes
N_dummy = 100  # Number of dummy scans

# Ex 1. Doubled angular increment (full circle instead of half circle)
delta = 2*np.pi / Nr  # Angular increment

ro_os = 2 # Readout oversampling
rf_spoiling_inc = 117  # RF spoiling increment

# Set system limits
system = pp.Opts(
    max_grad=28,
    grad_unit="mT/m",
    max_slew=150,
    slew_unit="T/m/s",
    rf_ringdown_time=20e-6,
    rf_dead_time=100e-6,
    adc_dead_time=10e-6,
)

seq = pp.Sequence(system)  # Create a new sequence object

# ======
# CREATE EVENTS
# ======
# Create alpha-degree slice selection pulse and gradient
rf, gz, _ = pp.make_sinc_pulse(
    apodization=0.5,
    duration=1e-3,
    flip_angle=alpha * np.pi / 180,
    slice_thickness=slice_thickness,
    system=system,
    time_bw_product=4,
    return_gz=True,
)
gz_reph = pp.make_trapezoid(channel="z", area=-gz.area / 2, system=system)

# Define other gradients and ADC events
deltak = 1 / fov

# Ex 1. Added division by 2 to gx flat_area and number of ADC samples
gx = pp.make_trapezoid(channel="x", flat_area=Nx/2 * deltak + deltak/4, flat_time=3.2e-3, system=system)
adc = pp.make_adc(num_samples=int(Nx/2*ro_os), duration=gx.flat_time, delay=gx.rise_time, system=system)

# Ex 1. Modified gx_pre to only compensate for the gradient area during ramp-up
gx_pre = pp.make_trapezoid(channel="x", area=-gx.rise_time * gx.amplitude / 2, duration=1e-3, system=system)

# Gradient spoiling
gx_spoil = pp.make_trapezoid(channel="x", area=0.5 * Nx * deltak, system=system)
gz_spoil = pp.make_trapezoid(channel="z", area=4 / slice_thickness, system=system)

# Calculate timing
# Ex 1. Removed delay_TE calculation
# delay_TE = (
#     np.ceil(
#         (
#             TE
#             - (pp.calc_duration(gz, rf) - pp.calc_rf_center(rf)[0] - rf.delay)
#             - pp.calc_duration(gx_pre, gz_reph)
#             - pp.calc_duration(gx) / 2
#             - pp.eps
#         )
#         / seq.grad_raster_time
#     )
#     * seq.grad_raster_time
# )
# assert delay_TE >= 0

delay_TR = (
    np.ceil(
        (
            TR
            - pp.calc_duration(gz, rf)
            - pp.calc_duration(gx_pre, gz_reph)
            - pp.calc_duration(gx)
            # Ex 1. Corrected delay_TR for removed delay_TE block
            # - delay_TE
        )
        / seq.grad_raster_time
    )
    * seq.grad_raster_time
)
assert delay_TR >= pp.calc_duration(gx_spoil, gz_spoil)

rf_phase = 0
rf_inc = 0

# ======
# CONSTRUCT SEQUENCE
# ======
for i in range(-N_dummy, Nr):
    # Set RF/ADC phase for RF spoiling, and increment RF phase
    rf.phase_offset = rf_phase / 180 * np.pi
    adc.phase_offset = rf_phase / 180 * np.pi

    rf_inc += rf_spoiling_inc
    rf_phase = (rf_inc + rf_phase) % 360

    # Slice-selective excitation pulse
    seq.add_block(rf, gz)

    # Slice rephaser and readout pre-phaser
    phi = delta * i
    seq.add_block(*pp.rotate(gx_pre, angle=phi, axis="z"), gz_reph)

    # Wait so readout is centered on TE
    # Ex 1. Removed delay_TE block
    # seq.add_block(pp.make_delay(delay_TE))

    # Readout gradient, rotated by `phi`
    if i >= 0:
        # Real scan, readout gradient + ADC object
        seq.add_block(*pp.rotate(gx, angle=phi, axis="z"), adc)
    else:
        # Dummy scan, do not add ADC object
        seq.add_block(*pp.rotate(gx, angle=phi, axis="z"))

    # GX/GZ spoiler gradient, and wait for TR
    seq.add_block(*pp.rotate(gx_spoil, angle=phi, axis="z"), gz_spoil, pp.make_delay(delay_TR))

# Timing check
ok, error_report = seq.check_timing()
if ok:
    print("Timing check passed successfully")
else:
    print("Timing check failed! Error listing follows:")
    print(''.join(error_report))

# ======
# VISUALIZATION
# ======
if plot:
    seq.plot()

# Print test report
print(seq.test_report())

# =========
# WRITE .SEQ
# =========
seq.set_definition(key="FOV", value=[fov, fov, slice_thickness])

if write_seq:
    seq.set_definition(key="Name", value="gre_rad")
    seq.write(seq_filename)

In [ ]:
k_traj_adc, k_traj, t_excitation, t_refocusing, t_adc = seq.calculate_kspace()

plt.figure()
plt.plot(k_traj[0],k_traj[1])
plt.plot(k_traj_adc[0],k_traj_adc[1],'.');

Calculate the distance between the last sample of the first readout and the last sample of the second readout, to check the Nyquist condition. This only works for linear steps in acquisition angle.

In [ ]:
if delta == 2*np.pi/Nr:
    k_traj_adc, k_traj, t_excitation, t_refocusing, t_adc = seq.calculate_kspace()
    deltak_edge = np.linalg.norm(k_traj_adc[:,adc.num_samples-1] - k_traj_adc[:,2*adc.num_samples-1])
    if deltak_edge >= deltak*1.001: # Allow for small error
        print(f'Not Nyquist sampled! {deltak / deltak_edge * 100:.1f}% ☹️')
    else:
        print(f'Nyquist sampled! {deltak / deltak_edge * 100:.1f}% 🥳')

**Animate trajectory**

In [ ]:
if animate_sequence:
    TR = t_excitation[1] - t_excitation[0]
    ani = animate(seq, show=False, dt=5e-4, time_range=[N_dummy*TR,N_dummy*TR + 2*TR], fps=5, show_progress=True)
    display(HTML(ani.to_html5_video()))

**Run MR-zero simulation**

The code below simulates the whole sequence, and shows the magnitude of the raw k-space profiles.

In [ ]:
# Simulate sequence
kdata = simulate_2d(seq, sim_size=[128,128], noise_level=0.5, n_coils=1).conj()

# Display raw data profiles
plt.figure()
plt.imshow(abs(kdata[0]))
plt.title('Magnitude of acquired raw data');

**Reconstruction**

To reconstruct non-Cartesian data, we need to use the non-uniform FFT (NUFFT). Here, we provide the `recon_nufft_2d` function to reconstruct non-Cartesian data using a NUFFT, and using the k-space trajectory calculated by `seq.calculate_kspace`.

***Implementation details***: Instead of using density-compensation, we used an iterative solver to solve `minimize ||NUFFT(rec) - kdata|| + lambda*||rec||`. The second term is a regularization term, that penalizes high values in voxels (L2 norm), which suppresses some of the most obvious aliasing artifacts.

There exist various density-compensation algorithms that would allow us to do this reconstruction with a single inverse NUFFT (i.e. `adj_NUFFT(kdata * density)`), assuming the data is sampled with a sufficient density.

In [ ]:
rec = recon_nufft_2d(kdata, seq, (Nx, Nx))

plot_nd(rec)

**Reconstruction: Compressed Sensing**

Since we are using iterative reconstruction, it is easy to add a more complicated regularization term. Here, we penalize the differences between neighouring voxels (called Total Variation regularization). The result of this is that the reconstruction will tend towards piecewise-constant values. Even in Nyquist-sampled acquisitions, a reconstruction like this will suppress noise.  The higher the regularization term `lambda_tv`, the stronger this effect will be.

***Implementation details:*** Internally this uses `sigpy.mri.app.TotalVariationRecon`, a ready made solution provided by `sigpy` to do such reconstructions easily. Although this is one of the most easy functions to use, there exist many other software packages to do iterative reconstructions! (e.g. BART).

In [ ]:
rec = recon_nufft_2d(kdata, seq, (Nx, Nx), lambda_tv=0.1)

plot_nd(rec)

# Exercises

Note: If you want the whole notebook to run a bit faster, consider setting `animate_sequence = False` and commenting out the Compressed Sensing reconstruction block for now.

## 1. Center-out radial

In order to achieve very short echo times, center-out radial trajectories immediately start measuring in the center of k-space. In this exercise we will modify the basic radial sequence into a center-out sequence.

As a first attempt, we will implement a center-out sequence that still samples only the flat part of the readout gradient.

- Modify the pre-phaser (`gx_pre`) gradient area such that the first sample of the readout is acquired in the center of k-space. (You can check this by evaluating `k_traj_adc[:,0]`, the k-space location of the first acquired sample).

- Modify the readout gradient (`gx`) and the ADC event to acquire half of a k-space line. Hint: You can check the resolution in the test report to see if it matches the prescribed resolution `fov/Nx` (4 mm with the settings provided).

- Since we are interested in achieving very short echotimes, remove the `delay_TE` block.

  - Optional: In the sequence test report you can observe that removing the `delay_TE` block invalidates the calculated TR value. Fix the `delay_TR` calculation accordingly.

- Since every line now only samples half of the k-space extent, we sample half a circle in k-space. Change `Nr` and `delta` such that a full k-space is sampled.

  - You can now lower `TR` by a fair amount, given that `TE` should now be much shorter, and less time is spent sampling. This can compensate for the extra time spent due to the increase in `Nr`.

- Check that the reconstructions look good!

- If you are continuing with part 2, consider saving a copy of this notebook in its current state, since we will be editing the sequence some more.

# 2. Optional: Ramp-sampling

In section 1 we still needed a small pre-phaser gradient (`gx_pre`) to compensate for the `gx` gradient area during ramp-up, which takes a little bit of time. In order to minimize `TE`, we will now use ramp sampling. This means that we acquire signal during the ramp-up of the readout gradient. And since we start acquisition at $k_0$, we no longer need `gx_pre`.

- Remove `gx_pre` from the sequence. Be mindful to not remove `gz_reph`.

Now note that we're not starting acquiring signal in the center of k-space. (You can check this by evaluating `k_traj_adc[:,0]`, the k-space location of the first acquired sample).

- Remove the delay from the ADC object and increase its duration to match the ramp-up and flat time of the readout gradient. Note that the minimum required delay for an ADC object is `system.adc_dead_time`. To make sure we start measuring at the start of the readout gradient, delay `gx` by this amount.

- It is very likely that at this point the timing check fails on the ADC object! That is because the dwell time (time per ADC sample) needs to align to the ADC raster time (`system.adc_raster_time`). This is a very common error to come across, for example, modifying `flat_time` in `gx` may trigger it. Always remember to check the timing of your sequences! We will fix this in the next step.

- The number of ADC samples is now too low, after all, we are sampling the ramp-up time. Increase the ADC samples proportionally to the new duration of the ADC object. Here, this happens to fix the timing issue (but this is not necessarily true in general).
  - Note that the number of ADC samples on Siemens scanners always needs to be a multiple of 2!

- Optional: We are almost there, but the slightly pedantic among you may have noticed that the resolution reported by the test report is slightly better than the 4mm we prescribed. This is because the area during the ramp-up of `gx` is added to the trajectory. You will need to lower the area of `gx` slightly, such that `(gx.rise_time/2 + gx.flat_time) * gx.amplitude` equals the desired area (what we set `flat_area` to before). This is not so trivial to do in general! Because once you modify amplitude, `rise_time` may change with the amplitude (unless you manually specify `rise_time`, but this may cause you to not use the maximum slew rate).

Finally, note that `TE` is now still limited by the duration of `gz_reph`. To really minimize `TE`, and achieve so-called ultra-short echotimes, we need to use non-selective RF pulses, which do not require slice-selection gradients. However, this can only be done for 3D imaging, which means that the trajectory needs to cover a 3D k-space! This is called a kooshball trajectory, and will be covered in the session tomorrow.

- Optional: If you have time left, you can try removing the slice-selection gradients to see how low an echotime you could theoretically achieve in a 3D sequence! Can you beat 0.54 ms? (Hint: the RF pulse is not optimal!)